<a href="https://colab.research.google.com/github/rudaruda/Python-SQLAlchemy-Experience/blob/main/BulkInsert_with_SQLAlchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste de Performance com SQLAlchemy

Importando as bibliotecas...

In [1]:
import time, psycopg2 #json, sys,  re,traceback,

from sqlalchemy import Column, Integer, BigInteger, String, Date, DateTime, Float, Boolean, create_engine, MetaData, Table, null
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.dialects.postgresql import insert

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Conectando no banco de dados PostgreSQL na esta GCP (Google Cloud Platform)

Praticamente um teste num cenário produtivo

In [2]:
DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASS = "00.00.00.000", "5432", "bd_xxx", "example_user", "***REMOVED***"
db = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
try:
  conn = db.connect()
  print ("Connected! on PostGreSQL")
except Exception as ex: print ("Error on PostGreSQL:", ex)

Connected! on PostGreSQL


Criando, tabela, sessão... setando metadata e automap...

In [21]:
# Check todos objetos do Banco
metadata = MetaData(db)

# Criando tabela "tb_test_test"
new_table = Table('tb_test_test', metadata,
  Column('Id', Integer, primary_key=True, autoincrement=True), Column('Date', Date), Column('Country', String) 
)
# Commit Create Table
new_table.create(checkfirst=True)

In [22]:
# Refletindo objetivos com AUTOMAP 
3 # !!! muito importante para o SQLAlchemy abstrair os objetos já existentes no Banco) !!! #
Base = automap_base()
Base.prepare(db, reflect=True)

In [23]:
# Set Table into variable, with automapper (mapper!)
tb_test = Base.classes.tb_test_test

# Create Session
Session = sessionmaker(bind=db)
s = Session()

# Estamos prontos para o que interessa

### 01. Executando a instrução de **Insert direto na conexão**
*(esse é o jeito mais tosco, mas beleza o importante é o teste)*

In [47]:
t0 = time.time()
n = 100
for i in range(n):
  db.execute("""insert into tb_test_test("Country", "Date") values ('Brasil','20210223')""")
print("SqlAlchemy [01]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [01]: Total time for 100 records 35.053895711898804 secs


### 02. Adicionando Objetos na Sessão **com flush** 
(para garantir cada registro)

In [48]:
t0 = time.time()
n = 100
for i in range(n):
  ins_tb_test = tb_test(Date='20210223',Country='Brasil')
  s.add(ins_tb_test)
  s.flush()
s.commit()
print("SqlAlchemy [02]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [02]: Total time for 100 records 11.971248149871826 secs


### 03. Adicionando Objetos na Sessão **sem flush** 
*(só com commit no final)*

In [49]:
t0 = time.time()
n = 100
for i in range(n):
  ins_tb_test = tb_test(Date='20210223',Country='Brasil')
  s.add(ins_tb_test)
s.commit()
print("SqlAlchemy [03]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [03]: Total time for 100 records 11.934319257736206 secs


### 04. Executando instrução de **Insert direto na sessão** 
*(e também com commit no final)*

In [50]:
t0 = time.time()
n = 100
for i in range(n):
  s.execute("""insert into tb_test_test("Country", "Date") values ('Brasil','20210223')""")
s.commit()
print("SqlAlchemy [04]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [04]: Total time for 100 records 11.909061431884766 secs


### 05. Adicionando todos os objetos de uma vez na sessão com **"add_all"**

In [51]:
Data = []
t0 = time.time()
n = 100
for i in range(n):
  ins_tb_test = tb_test(Date='20210223',Country='Brasil') 
  Data.append(ins_tb_test)
s.add_all(Data)
s.commit()
print("SqlAlchemy [05]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [05]: Total time for 100 records 11.92617416381836 secs


### 06. Adicionando todos os objetos de uma vez na sessão com **"bulk_save_objects"**

In [52]:
Data = []
t0 = time.time()
n = 100
for i in range(n):
  ins_tb_test = tb_test(Date='20210223',Country='Brasil') 
  Data.append(ins_tb_test)
s.bulk_save_objects(Data)
s.commit()
print("SqlAlchemy [06]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [06]: Total time for 100 records 11.886820316314697 secs


### 07. Adicionado instrução de **Insert em bloco na sessão**


In [53]:
Data = []
t0 = time.time()
n = 100
for i in range(n):
  Data.append({'Country':'Brasil','Date':'20210223'})
stmt=insert(tb_test,Data)
s.execute(stmt)
s.commit()
print("SqlAlchemy [07]: Total time for " + str(n) + " records " + str(time.time() - t0) + " secs")

SqlAlchemy [07]: Total time for 100 records 0.35940027236938477 secs


# Conclusão

> De acordo com os métodos comparados, o método que possui mais performance é o "7. Adicionado instrução de Insert em bloco na sessão" que realizou a ção em 0.3594 segundos (isso mesmo, menos de um segundo).

>Comparado ao tempo de moda de 11 segundos ele é 30x mais rápido. Isso por porque foi possível gerenciar a performance dos commits e somado a disponibilidade de arquitetura ORM é uma ótima saída.

> E sim, o SQLAlchemy tem vários métodos para fazer Update e Delete também. E sim, é possível testa-los, mas isso fica para a próxima ok?